# Práctica: Segmentación de clientes mayoristas (Wholesale customers – UCI)

## Contexto

El dataset **Wholesale customers** recoge información de clientes de un distribuidor mayorista.  
Cada fila es un cliente y las variables son:

- `Fresh`: gasto anual en productos frescos
- `Milk`: gasto anual en leche
- `Grocery`: gasto anual en alimentación general
- `Frozen`: gasto anual en congelados
- `Detergents_Paper`: gasto anual en detergentes y papel
- `Delicassen`: gasto anual en productos delicatessen
- `Channel`: canal del cliente (1 = Horeca: hoteles/restaurantes/cafés, 2 = Retail)
- `Region`: región (1 = Lisboa, 2 = Oporto, 3 = otras)

Objetivo: realizar una **segmentación de clientes** basada en los patrones de gasto y relacionarla con el canal y la región.

Aplicaremos:

1. **PCA** sobre los gastos para entender la estructura de la variabilidad.
2. **Clustering (k-means)** para obtener segmentos de clientes.
3. **Análisis de Correspondencias (CA)** para estudiar la relación entre segmentos y variables categóricas (`Channel`, `Region`).

---

## Instrucciones generales

- Completa las celdas que contienen comentarios `# TODO`.
- No elimines las celdas ni las preguntas en texto.
- Puedes añadir celdas adicionales para análisis extra.
- Entrega el notebook ejecutado, con todas las salidas visibles.


## 0. Importación de librerías

In [ ]:
# ============================================================
# 0. Importación de librerías
# ============================================================

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    adjusted_rand_score
)
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, classification_report

sns.set(style="whitegrid", context="notebook")

print("Librerías importadas correctamente.")


## 1. Carga del dataset y exploración inicial

El dataset original está disponible en el repositorio UCI como archivo CSV:

- Nombre: `Wholesale customers data.csv`

Si tienes conexión a internet en el entorno de Jupyter, puedes leerlo directamente desde la URL.  
Si no, deberás descargar el archivo previamente y leerlo desde disco.

En esta sección debes:

1. Cargar el dataset en un `DataFrame` de pandas.
2. Ver sus primeras filas.
3. Revisar tamaño, tipos de variables y valores faltantes.
4. Ver estadísticas descriptivas de las variables numéricas.
5. Explorar la distribución de `Channel` y `Region`.


In [ ]:
# ============================================================
# 1. Carga del dataset
# ============================================================

# TODO 1.1: Define la URL o ruta local del archivo CSV
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00292/Wholesale%20customers%20data.csv"

# TODO 1.2: Carga el CSV en un DataFrame llamado 'data' usando pd.read_csv(...)
data = 

# TODO 1.3: Muestra las primeras filas



In [ ]:
# TODO 1.4: Muestra dimensiones, información general y resumen estadístico de las variables numéricas

print("Dimensiones del dataset:", )

print("\nInformación general:")


print("\nResumen estadístico:")



In [ ]:
# TODO 1.5: Explora la distribución de 'Channel' y 'Region' (tablas de frecuencia y gráficos de barras)


### Preguntas de interpretación inicial

1. ¿Cuántos clientes y cuántas variables contiene el dataset?
2. ¿Existen valores faltantes en alguna variable?
3. ¿Cómo se reparten aproximadamente los clientes entre los canales (Horeca vs Retail)?
4. ¿Y entre las regiones?

*(Responde aquí con uno o varios párrafos.)*


## 2. Selección de variables de gasto y estandarización

Nos centraremos en las variables de gasto:

- `Fresh`, `Milk`, `Grocery`, `Frozen`, `Detergents_Paper`, `Delicassen`

Pasos:

1. Crear una matriz `X` con esas columnas.
2. Guardar `Channel` y `Region` por separado para usarlos más adelante como variables categóricas.
3. Estandarizar `X` (media 0, varianza 1) con `StandardScaler`.


In [ ]:
# ============================================================
# 2. Selección de variables y escalado
# ============================================================

# TODO 2.1: Define la lista de variables de gasto
spending_vars = 

# TODO 2.2: Crea X como matriz numpy con solo esas columnas
X = 

# TODO 2.3: Guarda Channel y Region como arrays para usarlos después
channel = 
region = 

print("Forma de X:", )

# TODO 2.4: Estandariza X usando StandardScaler
scaler = 
X_scaled = 



## 3. Análisis de Componentes Principales (PCA)

Objetivos:

1. Ajustar un PCA sobre `X_scaled`.
2. Calcular la varianza explicada por cada componente y la acumulada.
3. Elegir un número razonable de componentes.
4. Ajustar un PCA de 2 componentes y representar los clientes en el plano PC1–PC2.
5. Analizar los *loadings* (contribución de cada variable de gasto a PC1 y PC2).


In [ ]:
# ============================================================
# 3. PCA (todas las componentes)
# ============================================================

# TODO 3.1: Crea un objeto PCA (sin limitar n_components) y ajústalo a X_scaled
pca_full = 


# TODO 3.2: Extrae varianza explicada individual y acumulada
explained_var = 
cum_explained_var = 

print("Varianza explicada por componente:")



In [ ]:
# TODO 3.3: Representa el scree plot (varianza individual y acumulada)




In [ ]:
# ============================================================
# 3.1 PCA a 2 componentes y visualización
# ============================================================

# TODO 3.4: Ajusta un PCA de 2 componentes y proyecta X_scaled
pca_2 = 
X_pca2 = 

# TODO 3.5: Representa los clientes en el plano PC1–PC2 coloreando por Channel




In [ ]:
# TODO 3.6: Calcula los loadings de PC1 y PC2 y muéstralos en un DataFrame

loadings = 
loadings


### Preguntas de interpretación PCA

1. ¿Cuántas componentes principales se necesitan para explicar, por ejemplo, más del 80% de la varianza?
2. ¿Cómo interpretarías PC1 y PC2 en términos de tipos de gasto?
3. ¿Se aprecia algún patrón claro entre canal (Horeca/Retail) y la posición en el plano PC1–PC2?

*(Responde aquí.)*


## 4. Clustering k-means en el espacio PCA

Objetivo: identificar **segmentos de clientes** basados en los patrones de gasto.

Pasos:

1. Usar la proyección `X_pca2` (PC1 y PC2).
2. Aplicar **k-means** con un número de clusters `k` (por ejemplo, 3).
3. Visualizar los clusters en el plano PC1–PC2.
4. Calcular el **perfil de gasto medio** de cada cluster (media de cada variable de gasto en cada cluster).


In [ ]:
# ============================================================
# 4. Clustering k-means
# ============================================================

# TODO 4.1: Aplica k-means con k=3 sobre X_pca2
k = 
kmeans = 
clusters = 

print("Tamaño de cada cluster:", )


In [ ]:
# TODO 4.2: Representa los clusters en el plano PC1–PC2




In [ ]:
# TODO 4.3: Añade la etiqueta de cluster al DataFrame original y calcula perfil medio de gasto por cluster

df_clusters = 

cluster_profiles = 
cluster_profiles


In [ ]:
# TODO 4.4: Representa los perfiles de gasto medios por cluster (por ejemplo, como gráfico de barras)



### Preguntas de interpretación clustering

1. Describe brevemente cada cluster en términos de gasto medio en las distintas categorías.
2. ¿Puedes asociar cada cluster a algún tipo de cliente (por ejemplo, restaurante, supermercado, pequeño comercio...)?
3. ¿Te parece razonable el número de clusters elegido (k=3) o probarías otro valor?

*(Responde aquí.)*


## 5. Análisis de Correspondencias (CA) entre clusters y variables categóricas

Queremos estudiar la relación entre:

- Segmentos (clusters) y **Channel**.
- Segmentos (clusters) y **Region**.

Usaremos **Análisis de Correspondencias (CA)** sobre las tablas de contingencia:

1. `cluster × Channel`
2. `cluster × Region`

Implementaremos CA “a mano” con `numpy` usando:

- $N$: tabla de contingencia, con $n = \sum_{ij} N_{ij}$  
- $P = \dfrac{N}{n}$  
- Masas: $r = P\mathbf{1}$, \; $c = P^\mathsf{T}\mathbf{1}$  
- $D_r^{-1/2}, \; D_c^{-1/2}$  
- $S = D_r^{-1/2}\,(P - r c^\mathsf{T})\,D_c^{-1/2}$  
- SVD: $S = U \Sigma V^\mathsf{T}$  
- Filas: $F = D_r^{-1/2} U \Sigma$  
- Columnas: $G = D_c^{-1/2} V \Sigma$



In [ ]:
# ============================================================
# 5.1 Implementación genérica de CA
# ============================================================

def correspondence_analysis(contingency, n_components=2):
    """
    Implementación básica de Análisis de Correspondencias (CA).

    Parameters
    ----------
    contingency : pd.DataFrame
        Tabla de contingencia (filas x columnas).
    n_components : int
        Número de dimensiones a devolver.

    Returns
    -------
    row_coords : pd.DataFrame
        Coordenadas principales de las filas.
    col_coords : pd.DataFrame
        Coordenadas principales de las columnas.
    sing_vals : np.ndarray
        Valores singulares.
    """
    # TODO 5.1: Convierte contingency a N (float) y calcula n
    N = 
    n = 

    # TODO 5.2: Calcula P = N / n
    P = 

    # TODO 5.3: Calcula las masas de filas r y columnas c
    r = 
    c = 

    # TODO 5.4: Construye D_r^{-1/2} y D_c^{-1/2}
    D_r_inv_sqrt = 
    D_c_inv_sqrt = 

    # TODO 5.5: Calcula rc y la matriz S = D_r^{-1/2}(P - rc^T)D_c^{-1/2}
    rc = 
    S = 

    # TODO 5.6: Aplica SVD a S
    U, sing_vals, Vt = 

    # TODO 5.7: Calcula F y G (coordenadas principales) para las primeras n_components dimensiones
    dim = 
    F = 
    G = 

    row_coords = 
    col_coords = 

    return row_coords, col_coords, sing_vals


### 5.2 CA: cluster × Channel


In [ ]:
# ============================================================
# CA: cluster × Channel
# ============================================================

# TODO 5.8: Crea la tabla de contingencia cluster × Channel
contingency_cluster_channel = 

contingency_cluster_channel


In [ ]:
# TODO 5.9: Aplica correspondence_analysis a esta tabla
row_cc, col_cc, sv_cc = 

print("Valores singulares (cluster × Channel):")
print()


In [ ]:
# TODO 5.10: Representa filas (clusters) y columnas (Channel) en el plano CA

plt.figure(figsize=(7,6))

# Clusters (filas)


# Channel (columnas)



### Preguntas de interpretación CA cluster × Channel

1. ¿Qué clusters aparecen más cercanos a la categoría Horeca(1)?  
   ¿Y a Retail(2)?
2. ¿Algún cluster parece tener una mezcla más equilibrada de canales (cercano al origen)?
3. ¿Cómo encaja esto con los perfiles de gasto que describiste antes?

*(Responde aquí.)*


### 5.3 CA: cluster × Region


In [ ]:
# ============================================================
# CA: cluster × Region
# ============================================================

# TODO 5.11: Crea la tabla de contingencia cluster × Region
contingency_cluster_region = 

contingency_cluster_region


In [ ]:
# TODO 5.12: Aplica correspondence_analysis a esta tabla
row_cr, col_cr, sv_cr = 

print("Valores singulares (cluster × Region):")
print()


In [ ]:
# TODO 5.13: Representa clusters y regiones en el plano CA

plt.figure(figsize=(7,6))

# Clusters


# Regiones


### Preguntas de interpretación CA cluster × Region

1. ¿Hay algún cluster claramente asociado a alguna región (Lisboa, Oporto u otras)?
2. ¿Qué interpretación de negocio podrías darle a esta asociación geográfica?
3. ¿Se podría relacionar con tipos de cliente o estructura económica de las regiones?

*(Responde aquí.)*


## 7. Conclusiones finales

Integra aquí los resultados de:

- **PCA**: estructura de la variación en los patrones de gasto.
- **Clustering**: descripción de los segmentos de clientes.
- **CA**: relación entre segmentos y canal (Horeca/Retail), y entre segmentos y región.

### Guía de reflexión

1. Resume brevemente qué representa cada cluster (tipo de cliente).
2. Comenta qué categorías de gasto parecen más importantes para diferenciar segmentos.
3. Explica si los segmentos se corresponden claramente con un canal y/o región.
4. Evalúa si el número de clusters (k=3) es razonable o si propondrías cambiarlo.
5. Propón **al menos un análisis adicional** que podría hacerse:
   - Probar otros valores de `k` (y usar métodos como codo o silhouette).
   - Aplicar transformación logarítmica a los gastos y repetir el análisis.
   - Usar otros métodos de clustering (jerárquico, GMM, DBSCAN).

*(Escribe aquí tu síntesis final.)*
